# 2. Convolution and Pooling (10 - 10 pts)

### a) Write a function for convolution operation in the cell below. Write your own code with your own algorithm. It should have necessary error checks for matrix dimensions (consider multichannel matrices). Inputs must have the same structure as Tensorflow convolution operation and also the results must be same. Check [tf.nn.convolution](https://www.tensorflow.org/api_docs/python/tf/nn/convolution)

In [1]:
import tensorflow as tf #importy necessary packages
import numpy as np

In [0]:
def my_convolution(my_input, my_filter, my_padding, my_strides):
    # WRITE YOUR OWN CONVOLUTION IMPLEMENTATION HERE
    #  RULE CHECKS ##########
    if my_input.shape[3] != my_filter.shape[2]:   # Check number of channels' equality
        print("Input and filter must have the same number of channels")
        return -1

    if (my_strides[0] <= 0) or (my_strides[1] <= 0):  # values smaller than 1 are wrong.
        print("Number of strides must be greater than 0")
        return -1

    # my INPUT PADDING ####################
    if my_padding == "VALID":   # There is no padding in valid padding, output matrix dimensions are calculated here
        if (my_input.shape[1] >= my_filter.shape[0]) and (my_input.shape[2] >= my_filter.shape[1]): # In valid padding, FILTER SPATIAL SIZE CAN NOT BE LARGER THAN INPUT SPATIAL SIZE according to tf
            my_output = np.zeros([my_input.shape[0],
                                  int(np.ceil( (my_input.shape[1] - (my_filter.shape[0] -1)) / my_strides[0])), 
                                  int(np.ceil( (my_input.shape[2] - (my_filter.shape[1] -1)) / my_strides[1])),
                                  my_filter.shape[3]])
        else:
            print("FILTER SPATIAL SIZE CAN NOT BE LARGER THAN INPUT SPATIAL SIZE")
            return -1


    elif my_padding == "SAME":    # Same padding opt. implementation
        my_output = np.zeros([my_input.shape[0], int(np.ceil(my_input.shape[1] / my_strides[0])),
                              int(np.ceil(my_input.shape[2] / my_strides[1])),
                              my_filter.shape[3]])  # define an output array with proper dims.
        if my_input.shape[1] % my_strides[0] == 0:  #implementation of same padding according to given parameters
            row_pad= max((my_filter.shape[0] - my_strides[0]), 0) # calculation of row padding num.
        else:
            row_pad = max(my_filter.shape[0] - (my_input.shape[1] % my_strides[0]), 0)
        if my_input.shape[2] % my_strides[1] == 0:    # calculation of column padding num.
            column_pad = max((my_filter.shape[1] - my_strides[1]), 0)
        else:
            column_pad = max(my_filter.shape[1] - (my_input.shape[2] % my_strides[1]), 0)
        my_input_zeros =np.zeros([my_input.shape[0], int(my_input.shape[1] + row_pad) ,
                                  int(my_input.shape[2] + column_pad ), my_input.shape[3]])  # Create an zero valued matrix with calculated params.
        if row_pad != 0 and column_pad != 0:    # place the input in the zero valued new input vector. Below cases cover corner cases such as 0 padding.
            my_input_zeros[:, row_pad // 2:-(row_pad - (row_pad//2) ), column_pad//2 : -(column_pad - (column_pad//2)) ,:] = my_input
        else:
            if row_pad == 0 and column_pad == 0:
                my_input_zeros[:, row_pad // 2: , column_pad // 2: , :] = my_input
            elif row_pad == 0:
                my_input_zeros[:, row_pad // 2: , column_pad // 2: -(column_pad - (column_pad // 2)), :] = my_input
            else:
                my_input_zeros[:, row_pad // 2:-(row_pad - (row_pad // 2)), column_pad // 2: , :] = my_input

        my_input = my_input_zeros #Conv. part is written independent from padding type hence "my_input" variable should be the final output of padding part.
    else :
        return "PADDING SHOULD BE EITHER 'VALID' OR 'SAME'"

    #  CONVOLUTION ###############
    for batch_number in range(my_input.shape[0]): #for each image
        for output_channel in range(my_filter.shape[3]):  #for each filter
            for input_channel in range(my_input.shape[3]):  #for each channel of each input
                current_filter = my_filter[:, :, input_channel, output_channel]
                i = 0
                i_x = 0
                j = 0
                j_x = 0
                while i < my_input.shape[1]-my_filter.shape[0]+1: # pick a window from input
                    while j < my_input.shape[2]-my_filter.shape[1]+1:
                        window = my_input[batch_number, i: i + my_filter.shape[0], j: j + my_filter.shape[1], input_channel]
                        current_filter = current_filter.flatten()
                        window = window.flatten()
                        my_output[batch_number, i_x, j_x, output_channel] += np.correlate(window, current_filter) # multiply elemenwise and sum channelwise.
                        j += my_strides[1]
                        j_x += 1
                    i += my_strides[0]
                    i_x += 1
                    j = 0
                    j_x = 0


    return my_output

In [0]:
def tf_convolution(tf_input, tf_filter, tf_padding, tf_strides):
    
    sess = tf.Session()
    tf_function = tf.nn.convolution(input=tf_input, filter=tf_filter, padding=tf_padding, strides=tf_strides)
    tf_output = sess.run(tf_function)
    sess.close()
    
    return tf_output

In [5]:
import random   # this testbench part is highly parametrized, can be changed with the one given in original version. This provides cumulative error of randomized runs.
error = 0
for i in range (10):  #10 comparison
    number_of_channel = random.randint(1,10)  # #of channel must be same in both input and filter
    sample_input = np.random.random([random.randint(1,10), random.randint(1,200), random.randint(1,200), number_of_channel])
    padding_list = ["VALID", "SAME"]
    sample_padding = padding_list[random.randint(0,1)]
    if sample_padding == "VALID":
        sample_filter = np.random.random([random.randint(1,sample_input.shape[1]), random.randint(1,sample_input.shape[2]), number_of_channel, random.randint(1,10)])
    else:
        sample_filter = np.random.random([random.randint(1,200), random.randint(1,200), number_of_channel, random.randint(1,10)])   #If valid padding option used, filter spatial size can't be larger than
                                                                                                                                    # input spatial size in tensorflow. This condition is written in order to observe error
                                                                                                                                    #between tf and my_conv properly. my_conv, still checks this error explicitly.
    sample_strides = [random.randint(1,10),random.randint(1,10)]  #batch_size , channel_size and number of filters kept small in order to keep the calculation time short, not to put boundaries on test cases.

    
    print(sample_padding)
    tf_output = tf_convolution(sample_input,sample_filter,sample_padding,sample_strides)
    my_output = my_convolution(sample_input,sample_filter,sample_padding,sample_strides)

    print("input:")
    print(sample_input)
    print("filter:")
    print(sample_filter)
    print("TF output:")
    print(tf_output)
    print(tf_output.shape)
    print("MY OUTPUT")
    print(my_output)
    print(my_output.shape)
    print("cumulative DIFFERENCE :")
    error += np.sum(np.subtract(tf_output,my_output)) #cumulative error
    print(error)

SAME
input:
[[[[2.30491444e-01 9.26840283e-01 9.57948503e-01 3.42149662e-01
    9.89874465e-01]
   [1.32493101e-01 8.94115507e-01 5.46936464e-01 5.13325969e-01
    4.41379039e-01]
   [6.41601068e-01 6.59786607e-01 5.43925630e-01 7.91612565e-01
    3.79311333e-02]
   ...
   [5.47564284e-01 1.42252425e-01 3.32816282e-01 9.63878754e-01
    5.89165435e-01]
   [4.72818309e-01 6.78740902e-01 2.81934537e-03 1.95022819e-01
    1.78150637e-01]
   [9.29131217e-01 8.40549492e-01 1.09929449e-01 8.65420701e-02
    8.11664547e-01]]

  [[6.53966455e-01 3.59373450e-01 4.36854347e-01 6.03290056e-01
    9.32012773e-01]
   [3.31214333e-01 5.91541099e-01 7.50210717e-01 8.62473028e-02
    9.98586833e-01]
   [5.17432869e-01 3.99397403e-01 2.31055029e-01 1.64169158e-01
    2.17935096e-01]
   ...
   [2.27427893e-01 3.37062569e-01 4.16686402e-02 8.04095762e-01
    6.57814894e-01]
   [1.14615714e-01 3.36780163e-01 4.43207351e-01 9.99586231e-01
    2.24661605e-01]
   [5.99260393e-01 4.35998350e-02 1.55421209e-01

### b) Write a function for pooling operation in the cell below. Write your own code with your own algorithm. It should have necessary error checks for matrix dimensions (consider multichannel matrices). Inputs must have the same structure as Tensorflow pooling operation and also the results must be same. Check [tf.nn.pool](https://www.tensorflow.org/api_docs/python/tf/nn/pool)

In [0]:
def my_pooling(my_input, my_window_shape, my_pooling_type, my_padding, my_strides):
    # WRITE YOUR OWN POOLING IMPLEMENTATION HERE
    #  RULE CHECKS ##########

    if (my_strides[0] <= 0) or (my_strides[1] <= 0):  # meaning of rule checks are given in prints
        print("Number of strides must be greater than 0")
        return -1

    if (my_window_shape[0] < my_strides[0]) or (my_window_shape[0] < my_strides[0]):  
        print (" Stride > window shape is not supported in tensorflow due to inconsistency between CPU-GPU implementation")
        return -1

    # my INPUT PADDING ####################
    if my_padding == "VALID":
        my_output = np.zeros([my_input.shape[0],
                              int(np.ceil( (my_input.shape[1] - (my_window_shape[0] -1)) / my_strides[0])),
                              int(np.ceil( (my_input.shape[2] - (my_window_shape[1] -1)) / my_strides[1])),
                              my_input.shape[3]]) # There is no padding in valid padding, output matrix dimensions are calculated here

    elif my_padding == "SAME":
        my_output = np.zeros([my_input.shape[0], int(np.ceil(my_input.shape[1] / my_strides[0])),  int(np.ceil(my_input.shape[2] / my_strides[1])), my_input.shape[3]])
        if my_input.shape[1] % my_strides[0] == 0:  # calculation of row padding num.
            row_pad= max((my_window_shape[0] - my_strides[0]), 0)
        else:
            row_pad = max(my_window_shape[0] - (my_input.shape[1] % my_strides[0]), 0)
        if my_input.shape[2] % my_strides[1] == 0:  # calculation of column padding num.
            column_pad = max((my_window_shape[1] - my_strides[1]), 0)
        else:
            column_pad = max(my_window_shape[1] - (my_input.shape[2] % my_strides[1]), 0)
        my_input_zeros =np.zeros([my_input.shape[0], int(my_input.shape[1] + row_pad) ,
                                  int(my_input.shape[2] + column_pad ), my_input.shape[3]])  # Create an zero valued matrix with calculated params.
        if row_pad != 0 and column_pad != 0:  # place the input in the zero valued new input vector. Below cases cover corner cases such as 0 padding.
            my_input_zeros[:, row_pad // 2:-(row_pad - (row_pad//2) ), column_pad//2 : -(column_pad - (column_pad//2)) ,:] = my_input
        else:
            if row_pad == 0 and column_pad == 0:
                my_input_zeros[:, row_pad // 2: , column_pad // 2: , :] = my_input
            elif row_pad == 0:
                my_input_zeros[:, row_pad // 2: , column_pad // 2: -(column_pad - (column_pad // 2)), :] = my_input
            else:
                my_input_zeros[:, row_pad // 2:-(row_pad - (row_pad // 2)), column_pad // 2: , :] = my_input

        my_input = my_input_zeros
    else :
        print( "PADDING SHOULD BE EITHER 'VALID' OR 'SAME'")
        return -1

    #  p00ling ###############
    for batch_number in range(my_input.shape[0]): #for each input in batch
        for input_channel in range(my_input.shape[3]):  #for each channel of each input
            #current_filter = my_filter[:, :, input_channel, output_channel]
            i = 0
            i_x = 0
            j = 0
            j_x = 0
            while i < my_input.shape[1]-my_window_shape[0]+1: # run a pooling window over the image
                while j < my_input.shape[2]-my_window_shape[1]+1:
                    window = my_input[batch_number, i: i + my_window_shape[0], j: j + my_window_shape[1], input_channel]
                    if my_pooling_type == "MAX":  # max pooling type imp.
                        my_output[batch_number, i_x, j_x, input_channel] = np.max(window)
                    elif my_pooling_type == "AVG":  # avg pooling type imp. it differs for same and valid padding
                        if my_padding == "SAME":  #in same padding, tf.pool doesnt count the padded zeros in averaging, 
                        #a bruteforce but not the best soltn. was to only include non-zero elements in avg. calculation, this seems 
                        # applicable due to amount of zero valued cells in real img. Random trials prove thah difference doesnt go above 1e-8
                            my_output[batch_number, i_x, j_x, input_channel] = np.mean(window[np.nonzero(window)])
                        else: # for valid padding and avg. pooling
                            my_output[batch_number, i_x, j_x, input_channel] = np.mean(window)
                    else:
                        print("Pooling type must be either 'MAX' or 'AVG' ")  # check for pooling type.
                        return -1
                    j += my_strides[1]
                    j_x += 1
                i += my_strides[0]
                i_x += 1
                j = 0
                j_x = 0
    return my_output

In [0]:
def tf_pooling(tf_input, tf_window_shape, tf_pooling_type, tf_padding, tf_strides):
    
    sess = tf.Session()
    tf_function = tf.nn.pool(input=tf_input, window_shape=tf_window_shape, pooling_type=tf_pooling_type, padding=tf_padding, strides=tf_strides)
    tf_output = sess.run(tf_function)
    sess.close()
    
    return tf_output

In [8]:
import random
error = 0
for i in range (10):
    number_of_channel = random.randint(1,10)  # batch_size, number_of_channels kept small in order to keep calc. time small. no value boundary cond. implied.
    sample_input = np.random.random([random.randint(1,10), random.randint(1,200), random.randint(1,200), number_of_channel])
    sample_window_shape = [random.randint(1,sample_input.shape[1]), random.randint(1,sample_input.shape[2])]
    padding_list = ["VALID", "SAME"]
    sample_padding = padding_list[random.randint(0,1)]
    pooling_list = ["MAX", "AVG"]
    sample_pooling_type = pooling_list[random.randint(0,1)]
    sample_strides = [random.randint(1, sample_window_shape[0]),random.randint(1, sample_window_shape[1])]    # This parametrization done since tensorflow doesn't support strides > window_shape condition,
                                                                                              # my_pooling still checks this condition explicitly

    tf_output = tf_pooling(sample_input,sample_window_shape,sample_pooling_type,sample_padding,sample_strides)
    my_output = my_pooling(sample_input, sample_window_shape, sample_pooling_type, sample_padding, sample_strides)


    error += np.sum(np.subtract(tf_output,my_output))
    print ("Cumulative error :")
    print(error)
    print("")

Cumulative error :
0.0

Cumulative error :
-6.661338147750939e-16

Cumulative error :
-6.661338147750939e-16

Cumulative error :
-6.661338147750939e-16

Cumulative error :
2.886579864025407e-15

Cumulative error :
2.886579864025407e-15

Cumulative error :
2.886579864025407e-15

Cumulative error :
7.382983113757291e-15

Cumulative error :
7.382983113757291e-15

Cumulative error :
1.199040866595169e-14



## REFERENCES
[1] https://mmuratarat.github.io/2019-01-17/implementing-padding-schemes-of-tensorflow-in-python

[2] https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/nn/convolution

